In [1]:
import torch
import numpy as np
import PIL

print(torch.cuda.is_available())

True


In [2]:
!nvidia-smi

Tue Aug 20 20:51:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        Off | 00000000:03:00.0 Off |                  N/A |
| 30%   49C    P8              13W / 170W |     11MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms, models, datasets
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

In [4]:
RESCALE_SIZE = 224

In [5]:
train_transforms = transforms.Compose([
    transforms.Resize((RESCALE_SIZE, RESCALE_SIZE)),
    #transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x / 255),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [6]:
train_dir = "./journey-springfield/train/simpsons_dataset"

In [7]:
import math

image_datasets = datasets.ImageFolder(train_dir, train_transforms)

In [8]:
dataset_len = len(image_datasets)
p1 = (dataset_len / 100)
train_size = math.floor(p1 * 70)
test_size = dataset_len - train_size

In [9]:
train_dataset, val_dataset = torch.utils.data.random_split(image_datasets, [train_size, test_size])

In [10]:
len(train_dataset), len(val_dataset)

(14653, 6280)

In [11]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, num_workers=torch.cpu.device_count(),
    shuffle=True
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=128, num_workers=torch.cpu.device_count(),
)

loaders = {
    "train": train_dataloader,
    "valid": val_dataloader
}

In [12]:
model_resnet18 = models.resnet18(pretrained=True)
model_resnet18

/home/dmitry/Documents/programming/study/ml/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dmitry/Documents/programming/study/ml/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
for param in model_resnet18.parameters():
    param.requires_grad = False

In [14]:
for param in model_resnet18.avgpool.parameters():
    param.requires_grad = True
    
for param in model_resnet18.layer4.parameters():
    param.requires_grad = True

for param in model_resnet18.layer3.parameters():
    param.requires_grad = True

for param in model_resnet18.layer2.parameters():
    param.requires_grad = True

In [15]:
num_classes = 42
model_resnet18.fc = nn.Sequential(
    nn.Sequential(
        nn.Linear(512, 512),
        nn.BatchNorm1d(512),
        nn.Dropout(0.5),
        nn.ReLU()
    ),
    nn.Sequential(
        nn.Linear(512, 512),
        nn.Linear(512, num_classes)
    )
) 

In [16]:
import torch.optim as optim

optimizer = optim.Adam(
    (
        {
            "params": model_resnet18.layer2.parameters(),
            "lr": 1e-5
        },
        {
            "params": model_resnet18.layer3.parameters(),
            "lr": 1e-4
        },
        {
            "params": model_resnet18.layer4.parameters(),
            "lr": 1e-3
        },
        {
            "params": model_resnet18.avgpool.parameters()
        },
        {
            "params": model_resnet18.fc.parameters()
        }
    ), lr=1e-2)
scheduler = optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.5)

In [17]:
!rm -rf logs

In [18]:
from catalyst import dl
        
runner = dl.SupervisedRunner()

In [19]:
%%time

runner.train(
    model=model_resnet18,
    engine=dl.GPUEngine(),
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(),
    scheduler=scheduler,
    callbacks=[
        dl.CriterionCallback(input_key="logits", target_key="targets", metric_key="loss"),
        dl.BackwardCallback(metric_key="loss"),
        dl.OptimizerCallback(metric_key="loss"), 
        dl.AccuracyCallback(input_key="logits", target_key="targets"),
        dl.SchedulerCallback(),
        dl.PrecisionRecallF1SupportCallback(
            input_key="logits", target_key="targets", num_classes=num_classes, log_on_batch=False
        )
    ],
    loaders=loaders,
    num_epochs=35,
    verbose=True,
    logdir="logs/resnet18",
    load_best_on_end=True
)

1/35 * Epoch (train): 100%|███████████████████████████████████████████████████████████████| 115/115 [01:03<00:00,  2.71it/s, accuracy01=0.770, loss=1.158, lr=1.000e-05, momentum=0.900]/home/dmitry/Documents/programming/study/ml/venv/lib/python3.12/site-packages/accelerate/utils/dataclasses.py:480: FutureWarning: The `TPU` of `<enum 'DistributedType'>` is deprecated and will be removed in v1.0.0. Please use the `XLA` instead.
  warnings.warn(
1/35 * Epoch (train): 100%|███████████████████████████████████████████████████████████████| 115/115 [01:03<00:00,  1.81it/s, accuracy01=0.770, loss=1.158, lr=1.000e-05, momentum=0.900]


train (1/35) accuracy01: 0.5258308877710969 | accuracy01/std: 0.15454523830532205 | f1/_macro: 0.22291182125774606 | f1/_micro: 0.5258258879203338 | f1/_weighted: 0.5020046331343184 | loss: 2.5746156643220104 | loss/mean: 2.5746156643220104 | loss/std: 1.9057039754934 | lr: 1e-05 | momentum: 0.9 | precision/_macro: 0.22874558140463663 | precision/_micro: 0.5258308878727905 | precision/_weighted: 0.488232297672495 | recall/_macro: 0.22766767022939854 | recall/_micro: 0.5258308878727905 | recall/_weighted: 0.5258308878727906


1/35 * Epoch (valid): 100%|█████████████████████████████████████████████████████████████████| 50/50 [00:27<00:00,  1.83it/s, accuracy01=0.875, loss=0.728, lr=1.000e-05, momentum=0.900]


valid (1/35) accuracy01: 0.7103503184713377 | accuracy01/std: 0.03854817537873809 | f1/_macro: 0.3175745286215886 | f1/_micro: 0.7103453185065312 | f1/_weighted: 0.679332536233417 | loss: 1.2572756020126827 | loss/mean: 1.2572756020126827 | loss/std: 0.3985698012732361 | lr: 1e-05 | momentum: 0.9 | precision/_macro: 0.327234468087587 | precision/_micro: 0.7103503184713376 | precision/_weighted: 0.6705582770043308 | recall/_macro: 0.3303545560434812 | recall/_micro: 0.7103503184713376 | recall/_weighted: 0.7103503184713376
* Epoch (1/35) lr: 1e-05 | momentum: 0.9


2/35 * Epoch (train): 100%|███████████████████████████████████████████████████████████████| 115/115 [00:54<00:00,  2.09it/s, accuracy01=0.787, loss=0.907, lr=1.000e-05, momentum=0.900]


...

35/35 * Epoch (train): 100%|█| 115/115 [00:58<00:00,  1.96it/s, accura


train (35/35) accuracy01: 0.9997952635203792 | accuracy01/std: 0.0012480749527565861 | f1/_macro: 0.9734785677105904 | f1/_micro: 0.9997902637935165 | f1/_weighted: 0.9997599325081261 | loss: 0.000700712610426021 | loss/mean: 0.000700712610426021 | loss/std: 0.0022319816978492437 | lr: 1.5625e-07 | momentum: 0.9 | precision/_macro: 0.9713673053402723 | precision/_micro: 0.9997952637685116 | precision/_weighted: 0.9997407465193853 | recall/_macro: 0.9761287220847614 | recall/_micro: 0.9997952637685116 | recall/_weighted: 0.9997952637685117


35/35 * Epoch (valid): 100%|█| 50/50 [00:25<00:00,  1.98it/s, accuracy


valid (35/35) accuracy01: 0.9042993630573246 | accuracy01/std: 0.022966916377306367 | f1/_macro: 0.7467385441067136 | f1/_micro: 0.9042943630849704 | f1/_weighted: 0.9041229689100717 | loss: 0.7173662266154198 | loss/mean: 0.7173662266154198 | loss/std: 0.28672848235775306 | lr: 1.5625e-07 | momentum: 0.9 | precision/_macro: 0.7556374400473329 | precision/_micro: 0.9042993630573248 | precision/_weighted: 0.9057980623660875 | recall/_macro: 0.7484023778448328 | recall/_micro: 0.9042993630573248 | recall/_weighted: 0.904299363057325
* Epoch (35/35) lr: 7.8125e-08 | momentum: 0.9
Top models:
logs/resnet18/checkpoints/model.0035.pth	35.0000
CPU times: user 14min 27s, sys: 2min 42s, total: 17min 9s
Wall time: 47min 45s


/home/dmitry/Documents/programming/study/ml/venv/lib/python3.12/site-packages/catalyst/utils/torch.py:369: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.l

In [20]:
%load_ext tensorboard
%tensorboard --logdir logs

In [21]:
import os

class TestDataset(Dataset):
    def __init__(self, root, transforms):
        super().__init__()
        self.files = sorted(list(Path(root).rglob('*.jpg')), key=self.extract_image_number)
        self.transforms = transforms

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        x = self.load_sample(self.files[index])
        return self.transforms(x), 0

    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image

    def extract_image_number(self, path):
        filename = os.path.basename(path)
        return int(filename[3:-4])

In [22]:
test_transforms = transforms.Compose([
    transforms.Resize((RESCALE_SIZE, RESCALE_SIZE)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x / 255),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [23]:
test_dir = "./journey-springfield/testset/testset"
test_dataset = TestDataset(test_dir, test_transforms)

In [24]:
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=128
)

In [29]:
result = []

for prediction in runner.predict_loader(engine=dl.GPUEngine("cuda"), loader=test_dataloader):
    result += prediction['logits'].detach().cpu().numpy().argmax(1).tolist()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(image_datasets.classes)

In [ ]:
import pandas as pd

sample_submission_path = 'sample_submission.csv'
submission = pd.read_csv(sample_submission_path)
submission.head()

In [ ]:
submission['Expected'] = label_encoder.inverse_transform(result)
submission

In [ ]:
submission_path = 'submission.csv'
submission.to_csv(submission_path, index=None)